In [1]:
library(trustyai)
library(rJava)

In [2]:
center <- 10.0
epsilon <- 2.0

model <- trustyai::TestModels$getSumThresholdModel(center, epsilon)

In [3]:
features <- c(
  feature(name="x1", type="number", value=runif(n = 1, min = 0, max = 30)),
  feature(name="x2", type="number", value=runif(n = 1, min = 0, max = 30)),
  feature(name="x3", type="number", value=runif(n = 1, min = 0, max = 30)))

In [4]:
input <- prediction_input(features)
output <- c(as.list(as.list(model$predictAsync(J("java/util/List")$of(input))$get()$toArray())[[1]]$getOutputs())[[1]])

In [5]:
prediction <- simple_prediction(features, output)

In [6]:
saliencies <- lime(prediction, model)

In [7]:
cat(saliencies$asTable())

=== inside LIME Saliencies ==================
 Feature      Value |  Saliency  | Confidence
---------------------------------------------
   x1 =      16.156 |     0.023         0.000
   x2 =      14.810 |     0.022         0.000
---------------------------------------------
         Prediction |     0.000              

# Simple linear regression

In [8]:
data(trees)
head(trees)

,Girth,Height,Volume
,<dbl>,<dbl>,<dbl>
1,8.3,70,10.3
2,8.6,65,10.3
3,8.8,63,10.2
4,10.5,72,16.4
5,10.7,81,18.8
6,10.8,83,19.7


In [9]:
regression <- lm(Volume ~ Girth * Height, data = trees)

In [10]:
summary(regression)


Call:
lm(formula = Volume ~ Girth * Height, data = trees)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.5821 -1.0673  0.3026  1.5641  4.6649 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  69.39632   23.83575   2.911  0.00713 ** 
Girth        -5.85585    1.92134  -3.048  0.00511 ** 
Height       -1.29708    0.30984  -4.186  0.00027 ***
Girth:Height  0.13465    0.02438   5.524 7.48e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.709 on 27 degrees of freedom
Multiple R-squared:  0.9756,	Adjusted R-squared:  0.9728 
F-statistic: 359.3 on 3 and 27 DF,  p-value: < 2.2e-16


In [11]:
prediction_fn <- function(df) {
  return(predict(regression, df))
}

In [12]:
input <- data.frame(Girth = 18.2, Height = 72)
output <- prediction_fn(input)

In [13]:
l_input <- list("Girth"=18.2, "Height"=72)
result <- as.double(prediction_fn(l_input))

In [14]:
model <- Model(prediction_fn)

In [15]:
features <- c(
  feature(name="Girth", type="number", value=18.2),
  feature(name="Height", type="number", value=72.0))

output <- c(create_output("Volume", result))

prediction <- simple_prediction(features, output)

saliencies <- lime(prediction, model)

In [16]:
cat(saliencies$asTable())

=== Volume LIME Saliencies ===================
  Feature      Value |  Saliency  | Confidence
----------------------------------------------
 Girth =      18.200 |  -121.838         0.000
----------------------------------------------
          Prediction |    45.881              